In [4]:
import pandas as pd # 0.21.0
import numpy as np
from functools import reduce
from time import time

import scipy
from scipy import stats
import matplotlib.pyplot as plt

import os
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import quandl

In [5]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)

In [6]:
class SequenceDataset(Dataset):

    """
    Custom dataset to convert features to inputs of sequences.
    """

    def __init__(self, csv_file, root_dir, time_steps, transform=None):
        self._steps = time_steps
        self._transform = transform
        self._frames = pd.read_csv(os.path.join(root_dir, csv_file))
        
        self._y = self._frames.pop("y")
        self._X = self._frames

    def __len__(self):
        return len(self._frames.index) - self._steps

    def __getitem__(self, idx):
        index = idx + self._steps
        # Normalize over sequence (TODO: Find a better way to handle)
        X = self._X.iloc[index-self._steps:index]
        X_mean = X.apply(np.mean,axis=0)
        X_std = X.apply(np.std,axis=0)
        X_normalized = (X - X_mean)/(X_std+1e-10)
        sequence = torch.from_numpy(X_normalized.as_matrix())
        
        label = torch.from_numpy(np.array([self._y.iloc[index]]))
        return {'sequence': sequence, 'label': label}
    
    

In [7]:
train_data = SequenceDataset('train_data.csv','.',25)
test_data = SequenceDataset('test_data.csv','.',25)

train_loader = DataLoader(train_data, batch_size=1, shuffle=False, num_workers=4)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=4)



In [8]:
class SimpleLSTM(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, batch_size, steps):
        super(SimpleLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size_1, self.hidden_size_2, self.hidden_size_3 = hidden_sizes
        self.batch_size = batch_size
        self.steps = steps
        
        # Try preprocess with FC layer
        # https://danijar.com/tips-for-training-recurrent-neural-networks/
        self.fc_pre = nn.Linear(self.input_size, self.hidden_size_1)
        self.rnn_1 = nn.LSTM(self.hidden_size_1, self.hidden_size_2, num_layers=2, dropout=0.5, batch_first=True)
        self.rnn_2 = nn.LSTM(self.hidden_size_2, self.hidden_size_3, num_layers=2, dropout=0.5, batch_first=True)
        self.rnn_3 = nn.LSTM(self.hidden_size_3, hidden_size=self.hidden_size_3, num_layers=2, dropout=0.5, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(self.hidden_size_3, 2),
            nn.Sigmoid()
        )
        
        """
        # Consider learning the hidden initialization
        self.h_0 = nn.Parameter(
            torch.randn(2, self.batch_size, self.hidden_size).type(torch.FloatTensor),
            requires_grad=True
        )
        self.c_0 = nn.Parameter(
            torch.randn(2, self.batch_size, self.hidden_size).type(torch.FloatTensor),
            requires_grad=True
        )
        """
        

    def forward(self, x, train=True):
        out = self.fc_pre(x)
        out, _ = self.rnn_1(out)
        out, _ = self.rnn_2(out)
        out, _ = self.rnn_3(out)
        # We want the out of the last step (batch, step, out)
        return self.fc(out[:,-1,:])

In [10]:
model = SimpleLSTM(input_size=190, hidden_sizes=[150,100,50], batch_size=1, steps=25)
optimizer =  torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

In [ ]:
model.train()
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = []
    for i, batch in enumerate(train_loader):
        inputs = batch['sequence']
        labels = batch['label']
        
        inputs, labels = Variable(inputs.type(torch.FloatTensor)), Variable(labels.type(torch.LongTensor).squeeze(1))
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        # clip gradients
        torch.nn.utils.clip_grad_norm(model.parameters(), 0.75)
        
        optimizer.step()
    

        # print statistics
        running_loss.append(loss.data[0])
    
    print('Epoch: {}  loss: {}'.format(
          epoch + 1, np.mean(np.array(running_loss))))


print('Finished Training')

Epoch: 1  loss: 0.6943758453972497
Epoch: 2  loss: 0.69370237305878
Epoch: 3  loss: 0.6931226869822272
Epoch: 4  loss: 0.6932854015463833
Epoch: 5  loss: 0.6930906610844999
Epoch: 6  loss: 0.6932126847483132
Epoch: 7  loss: 0.6928666956365361
Epoch: 8  loss: 0.6928274171889444
Epoch: 9  loss: 0.6928586278746326
Epoch: 10  loss: 0.692744545525468
Epoch: 11  loss: 0.6926654298593582
Epoch: 12  loss: 0.6927360055225144
Epoch: 13  loss: 0.6927462225258532
Epoch: 14  loss: 0.6926856611668158
Epoch: 15  loss: 0.6926621227967934
Epoch: 16  loss: 0.6927105606000352
Epoch: 17  loss: 0.69268885133624
Epoch: 18  loss: 0.692669067950269
Epoch: 19  loss: 0.6926576346792304
Epoch: 20  loss: 0.6927270500241588
Epoch: 21  loss: 0.6927014498111329
Epoch: 22  loss: 0.692469694430566
Epoch: 23  loss: 0.6930685638295714
Epoch: 24  loss: 0.6929333545833049
Epoch: 25  loss: 0.6926589706266875
Epoch: 26  loss: 0.6930278428554825
Epoch: 27  loss: 0.6927133353102475
Epoch: 28  loss: 0.692745605670255
Epoch: 29

In [95]:
def evaluate(model, loader):
    model.eval()
    loss_sum = 0
    acc_sum = 0
    for i, batch in enumerate(loader):
        inputs = batch['sequence']
        labels = batch['label']
        inputs = Variable(inputs.type(torch.FloatTensor), volatile=True)
        labels = Variable(labels.type(torch.LongTensor).squeeze(1), volatile=True)
        
        output = model(inputs)
        loss = loss_fn(output, labels)
        loss_sum += loss.data[0]

        predict = output.data.max(1)[1]
        acc = predict.eq(labels.data).cpu().sum()
        acc_sum += acc
    return loss_sum / len(loader), acc_sum / len(loader)

In [96]:
print(evaluate(model, test_loader))

(0.6886351095732822, 0.5591397849462365)


In [207]:
score = Variable(torch.randn(10,2))
target = Variable((torch.rand(10)>0.5).long())
lfn1 = torch.nn.CrossEntropyLoss()
lfn2 = torch.nn.BCELoss()
print(lfn1(score,target), lfn2(torch.nn.functional.softmax(score)[:,1],target.float()))

Variable containing:
 0.8571
[torch.FloatTensor of size 1]
 Variable containing:
 0.8571
[torch.FloatTensor of size 1]



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """


In [165]:
score

Variable containing:
-1.0032 -0.6803
 1.2381  0.0608
-0.8555  0.6880
 0.1313 -1.2839
-0.4654 -0.2737
-0.7864  0.7487
 1.4117  0.9339
 1.9578 -0.0274
-1.0772  2.5011
 0.2887 -0.4335
[torch.FloatTensor of size 10x2]

In [166]:
target

Variable containing:
 0
 1
 0
 1
 0
 1
 1
 0
 1
 1
[torch.LongTensor of size 10]